In [69]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.stats import t
from scipy.optimize import curve_fit
import astropy.units as U

parent_dir=os.path.dirname(os.getcwd())
data = pd.read_csv(os.path.join(parent_dir, 'brightnes', 'CO10.dat'), sep="\t")
data


,#redshift,Luminosity(K km s-1 pc2),Sigma(Jy),luminosity distance(Mpc)
0,0.053,1.445440e+09,0.054663,238.578180
1,0.064,8.317638e+08,0.021454,290.390050
2,0.068,1.548817e+09,0.035318,309.421719
3,0.078,2.137962e+09,0.036875,357.442552
4,0.082,3.548134e+09,0.055268,376.826406
...,...,...,...,...
62,2.846,8.709636e+10,0.001149,24410.653651
63,3.094,2.398833e+11,0.002754,27001.725826
64,3.532,1.174898e+11,0.001086,31655.448098
65,3.796,1.819701e+11,0.001499,34502.645233


In [70]:
def read_line_emission_csv(path_line_emission_csv):
    """
    Read the csv file in which are stored the line emission's rest frequency.
    
    Parameter: 
    path_line_emission_csv (str): Path to file.csv within there are the line emission's rest frequency.
    
    Return:
    pd.DataFrame : Dataframe with line names and rest frequencies.
    """
    db_line = pd.read_csv(path_line_emission_csv, sep = "\t")
    return db_line
db_line = read_line_emission_csv(os.path.join(parent_dir, 'metadata', 'line_emission.csv'))
db_line


,line_emission,rest_freq(GHz)
0,CO(1-0),115.27100
1,CO(2-1),230.53800
2,H(1-0),14.40500
3,H(2-1),28.20536
4,O(3-2),495892.30000


In [71]:
def get_line_rest_frequency(line_name):
    """
    Takes as input the line name and extract from the dataframe the rest frequency of said line

    Parameter:
    line_name (str): The line name of the wanted rest frequency.

    Return:
    rest_frequency (float): The rest frequency of the selected line name.
    """
    db_line = read_line_emission_csv(os.path.join(parent_dir, 'metadata', 'line_emission.csv'))
    if line_name not in db_line["line_emission"].to_list():
        raise ValueError(f"The line name, '{line_name}', cannot be found in the dataframe.")
    rest_frequency = db_line[db_line["line_emission"]==line_name]["rest_freq(GHz)"]
    return rest_frequency
line_name = input("Chose the line emission name among the following lines: CO(1-0), CO(2-1), H(1-0), H(2-1), O(3-2)")
rest_frequency = get_line_rest_frequency(line_name)
rest_frequency


1    230.538
Name: rest_freq(GHz), dtype: float64

In [72]:
def luminosity_to_jy(velocity, data, line_name):
        """
        This function takes as input a pandas db containing luminosities in K km s-1 pc2, redshifts, and luminosity distances in Mpc, 
        and returns the brightness values in Jy, for the chosen line emission name.
        
        Parameters:
        velocity (float): The velocity dispersion assumed for the line (Km s-1).
        data (pandas.DataFrame): A pandas DataFrame containing the data: Luminosity(K km s-1 pc2),#redshift,luminosity distance(Mpc).
        line_name (str): The choosen line emission name.

        Output:
        sigma: numpy.ndarray: An array of brightness values in Jy.
        """
        def sigma_CO10(df, velocity, rest_frequency):
                alpha = 3.255e7
                return df['Luminosity(K km s-1 pc2)'] * ( (1 + df['#redshift']) * rest_frequency **2) / (alpha * velocity * (df['luminosity distance(Mpc)']**2))
        def sigma_CO21(df, velocity, rest_frequency):
                alpha = 3.255e7
                return df['Luminosity(K km s-1 pc2)'] * ( (1 + df['#redshift']) * rest_frequency **2) / (alpha * velocity * (df['luminosity distance(Mpc)']**2))
        def sigma_H10(df, velocity, rest_frequency):
                alpha = 3.255e7
                return df['Luminosity(K km s-1 pc2)'] * ( (1 + df['#redshift']) * rest_frequency **2) / (alpha * velocity * (df['luminosity distance(Mpc)']**2))
        def sigma_H21(df, velocity, rest_frequency):
                alpha = 3.255e7
                return df['Luminosity(K km s-1 pc2)'] * ( (1 + df['#redshift']) * rest_frequency **2) / (alpha * velocity * (df['luminosity distance(Mpc)']**2))
        def sigma_O32(df, velocity, rest_frequency):
                alpha = 3.255e7
                return df['Luminosity(K km s-1 pc2)'] * ( (1 + df['#redshift']) * rest_frequency **2) / (alpha * velocity * (df['luminosity distance(Mpc)']**2))
        rest_frequency = get_line_rest_frequency(line_name)
        function = {
                "CO(1-0)": (data, sigma_CO10),
                "CO(2-1)": (data, sigma_CO21),
                "H(1-0)": (data, sigma_H10),
                "H(2-1)": (data, sigma_H21),
                "O(3-2)": (data, sigma_O32),
                }
        sigma = data.apply(lambda row: function.get(line_name)[1](row, velocity, rest_frequency), axis=1)
        sigma = np.concatenate(sigma.to_numpy())                                                                                                
        return sigma
        
sigma = luminosity_to_jy(200,data,line_name)
sigma

array([0.21830867, 0.08568054, 0.1410502 , 0.14726865, 0.22072354,
       0.08283713, 0.02276395, 0.05999618, 0.04472609, 0.03575144,
       0.00284277, 0.00554297, 0.00775081, 0.00314952, 0.00406873,
       0.0038317 , 0.00571419, 0.00875031, 0.01191368, 0.00407761,
       0.00106755, 0.00501157, 0.00994564, 0.00206958, 0.0046105 ,
       0.07611312, 0.01175759, 0.00432717, 0.00387004, 0.00393098,
       0.00193678, 0.06981659, 0.00863407, 0.00432537, 0.00951724,
       0.0072895 , 0.00630848, 0.00445805, 0.00488815, 0.00706553,
       0.00262509, 0.00312794, 0.00819131, 0.00296239, 0.01044887,
       0.00400741, 0.0053654 , 0.00374744, 0.00621423, 0.00414905,
       0.00365486, 0.00209753, 0.00242841, 0.00729104, 0.00164303,
       0.00728746, 0.00647219, 0.00168253, 0.00332962, 0.00624194,
       0.00528599, 0.00409835, 0.00458939, 0.01099686, 0.00433807,
       0.0059852 , 0.00418469])

In [73]:
def exponential_func(x, a, b):
        """
        Exponential function used to fit the data.
        """
        return a * np.exp(-b * x)

In [ ]:
def sample_from_given_line_name(n, velocity, line_name, min_redshift, max_redshift):
    """
    Gives n samples of brightness values based on an exponential fit to the data.

    Parameters:
    n (int) : Number of samples to generate.
    line_name (str) : The line emission name picked.
    velocity (float) :  Velocity dispersion assumed for the given line.
    min_redshift, max_redshift (float) : The redshift range 

    Return:
    pd.DataFrame: A DataFrame into which sampled brightness values and corresponding redshifts are crammed.
    """
    # Generate evenly spaced redshifts for sampling
    np.random.seed(42)
    sampled_redshifts = np.linspace(min_redshift, max_redshift, n)
    params = {
            "CO(1-0)": (popt, pcov),
            "CO(2-1)": (popt, pcov),
            "H(1-0)": (popt, pcov),
            "H(2-1)": (popt, pcov),
            "O(3-2)": (popt, pcov),
}
    popt, pcov = params.get(line_name)
    # Use the fitted parameters to calculate the sampled brightness values
    sampled_sigma = exponential_func(sampled_redshifts, *popt) + np.min(sigma)
    plt.scatter(sampled_redshifts, sampled_sigma, label='Polinomial Fit')
    plt.xlabel('Redshift')
    plt.ylabel('Brightness (Jy)')
    plt.legend()
    plt.show()
    # Return the sampled brightness values and the corresponding redshifts
    return pd.DataFrame(zip(sampled_redshifts, sampled_sigma), columns=['Redshift', 'Brightness(Jy)'])
    
